In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from FUCCIDataLoader import FUCCIDataset
from UnetModel import UNet
from Train import train
from torchvision import transforms
import time


train_dataset = FUCCIDataset(root_dir="leonor", source_channels=0, target_channels=(0, 1), transform=transforms.RandomCrop(256)) #Do we need to crop? Guess also some data augmentation here would be nice
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=8) 

load_path = "/group/dl4miacourse/projects/FUCCI/leonor/Models/model1.pt"


### Declare the U-Net and its parameters

model = UNet(depth=2, in_channels=1, out_channels=2)
model.load_state_dict(torch.load(load_path))

#TODO tensoboard logs
#logger = SummaryWriter(f"unet_runs/{model_name}")


        
#After confirming that runs at least one epoch, go to terminal and nvdia-smi to be sure it's running in the GPU

In [ ]:
# run validation after training epoch
def validate(
    model,
    loader,
    loss_function,
    #metric,
    step=None,
    tb_logger=None,
    device=None,
):
    if device is None:
        # You can pass in a device or we will default to using
        # the gpu. Feel free to try training on the cpu to see
        # what sort of performance difference there is
        if torch.cuda.is_available():
            device = torch.device("cuda")
        else:
            device = torch.device("cpu")

    # set model to eval mode
    model.eval()
    model.to(device)

    # running loss and metric values
    val_loss = 0
    #val_metric = 0

    x_list = []
    y_list = []
    pred_list = []

    # disable gradients during validation
    with torch.no_grad():
        # iterate over validation loader and update loss and metric values
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            prediction = model(x)
            # We *usually* want the target to be the same type as the prediction
            # however this is very dependent on your choice of loss function and
            # metric. If you get errors such as "RuntimeError: Found dtype Float but expected Short"
            # then this is where you should look.
            if y.dtype != prediction.dtype:
                y = y.type(prediction.dtype)
            val_loss += loss_function(prediction, y).item()
            #val_metric += metric(prediction > 0.5, y).item()

            x_list.append(x.cpu())
            y_list.append(y.cpu())
            pred_list.append(prediction.cpu())

        



    # normalize loss and metric
    val_loss /= len(loader)
    #val_metric /= len(loader)

    print(
        "\nValidate: Average loss: {:.4f}\n".format(
            val_loss
        )
    )

    return x_list, y_list, pred_list

In [ ]:
loss_function: torch.nn.Module = torch.nn.MSELoss()
x_list, y_list, pred_list = validate(model, train_loader, loss_function)

In [ ]:
y_list[0].shape

In [ ]:
y_list[0].shape

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
batch_idx = np.random.randint(len(x_list))
item_idx = np.random.randint(len(x_list[batch_idx]))
print(batch_idx, item_idx)

_, ax = plt.subplots(figsize=(15,3),ncols=5)
ax[0].imshow(x_list[batch_idx][item_idx,0])
ax[1].imshow(y_list[batch_idx][item_idx,0])
ax[2].imshow(y_list[batch_idx][item_idx,1])
ax[3].imshow(pred_list[batch_idx][item_idx,0])
ax[4].imshow(pred_list[batch_idx][item_idx,1])

plt.savefig("p.png")